In [1]:
import numpy as np 
import pandas as pd

In [2]:
train=pd.read_csv('C:/Users/sai/Desktop/titanic_data/train.csv')
test=pd.read_csv("C:/Users/sai/Desktop/titanic_data/test.csv")
gs=pd.read_csv("C:/Users/sai/Desktop/titanic_data/gender_submission.csv")

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
for col in train.columns:
    print(train[col].value_counts(),'\n')


1      1
599    1
588    1
589    1
590    1
      ..
301    1
302    1
303    1
304    1
891    1
Name: PassengerId, Length: 891, dtype: int64 

0    549
1    342
Name: Survived, dtype: int64 

3    491
1    216
2    184
Name: Pclass, dtype: int64 

Braund, Mr. Owen Harris                     1
Boulos, Mr. Hanna                           1
Frolicher-Stehli, Mr. Maxmillian            1
Gilinski, Mr. Eliezer                       1
Murdlin, Mr. Joseph                         1
                                           ..
Kelly, Miss. Anna Katherine "Annie Kate"    1
McCoy, Mr. Bernard                          1
Johnson, Mr. William Cahoone Jr             1
Keane, Miss. Nora A                         1
Dooley, Mr. Patrick                         1
Name: Name, Length: 891, dtype: int64 

male      577
female    314
Name: Sex, dtype: int64 

24.00    30
22.00    27
18.00    26
19.00    25
28.00    25
         ..
36.50     1
55.50     1
0.92      1
23.50     1
74.00     1
Name: Age, Length

In [4]:
def clean(train):
    train=train.drop(['Ticket','Cabin','Name'],axis=1)
    cols=['SibSp','Parch','Fare','Age']
    for col in cols:
        train[col].fillna(train[col].median(),inplace=True)
        
    train.Embarked.fillna('U',inplace=True)  
    return train
train=clean(train)
test=clean(test)

In [5]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [6]:
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


In [7]:
from sklearn import preprocessing
l=preprocessing.LabelEncoder()

cols=['Sex','Embarked']

for col in cols:
    train[col]=l.fit_transform(train[col])
    test[col]=l.fit_transform(test[col])
    print(l.classes_)
train.head()    

['female' 'male']
['C' 'Q' 'S']


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,2
1,2,1,1,0,38.0,1,0,71.2833,0
2,3,1,3,0,26.0,0,0,7.9250,2
3,4,1,1,0,35.0,1,0,53.1000,2
4,5,0,3,1,35.0,0,0,8.0500,2


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y=train['Survived']
X=train.drop('Survived',axis=1)

X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
clf=LogisticRegression(random_state=0,max_iter=1000).fit(X_train,y_train)

In [10]:
y_val

709    1
439    0
840    0
720    1
39     1
      ..
433    0
773    0
25     1
84     1
10     1
Name: Survived, Length: 179, dtype: int64

In [11]:
predictions=clf.predict(X_val)
from sklearn.metrics import accuracy_score
accuracy_score(y_val,predictions)

0.8100558659217877

In [12]:
submission_pred=clf.predict(test)
accuracy_score(gs['Survived'], submission_pred)

0.9234449760765551

In [13]:
df=pd.DataFrame({'PassengerId':test.PassengerId,'Survived':submission_pred})

In [14]:
# df.to_csv('C:/Users/sai/Desktop/titanic_data/submission.csv', index=False)